<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


### Assignment for Week 3

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Author : Muhammad Adisatriyo Pratama

References : [Coursera Applied Data Science Capstone](https://www.coursera.org/learn/applied-data-science-capstone)

December 2020

Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [1]:
!pip install folium

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !pip install -c folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Problem 1

Importing data from Wikipedia : [Postal Codes of Canada](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# make a requests
wiki_url = requests.get(url)
wiki_url

<Response [200]>

wiki_url returns Response 200 wich means connection is established

In [4]:
# Read html from wiki_url using pandas library
html_wiki = pd.read_html(wiki_url.text)

# check type
print(f'Type of html_wiki : {type(html_wiki)}')

# length of html_wiki
print(f'Length of html_wiki : {len(html_wiki)}')

Type of html_wiki : <class 'list'>
Length of html_wiki : 3


In [5]:
# Check first few data from list
html_wiki[0:5]

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 5           M6A        North York   
 6           M7A  Downtown Toronto   
 7           M8A      Not assigned   
 8           M9A         Etobicoke   
 9           M1B       Scarborough   
 10          M2B      Not assigned   
 11          M3B        North York   
 12          M4B         East York   
 13          M5B  Downtown Toronto   
 14          M6B        North York   
 15          M7B      Not assigned   
 16          M8B      Not assigned   
 17          M9B         Etobicoke   
 18          M1C       Scarborough   
 19          M2C      Not assigned   
 20          M3C        North York   
 21          M4C         East York   
 22          M5C  Downtown Toronto   
 23          M6C              York   
 24          M7C      Not assigned   
 25         

We want first table wich is consist of three columns: PostalCode, Borough, and Neighborhood.

In [10]:
# We want first table on the list
df = html_wiki[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


 Ignore cells with a borough that is **Not assigned**.

In [11]:
# Dropping 'Not assigned' borough
df[df['Borough'] == 'Not assigned']


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned
16,M8B,Not assigned,Not assigned
19,M2C,Not assigned,Not assigned
24,M7C,Not assigned,Not assigned
25,M8C,Not assigned,Not assigned
28,M2E,Not assigned,Not assigned


In [13]:
df = df[df['Borough'] != 'Not assigned']
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

So we will group the records based on Postal Code

In [23]:
# Group by 'Postal Code'
clean_df = df.groupby(['Postal Code']).head()
clean_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
# Check 'Not assigned' Neighbourhood
clean_df[clean_df['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


There is no  *'Not assigned'*  Neighbourhood left

#### Resetting index for clean_df to start from 0

In [25]:
# resetting index for clean_df to start from 0
clean_df = clean_df.reset_index(drop=True)
clean_df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Shape of clean_df containing information about Neighbourhood in Toronto

In [26]:
clean_df.shape

(103, 3)

clean_df contains 103 rows and 3 columns

## Problem 2
### Discovering latitude and longitude

We were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to add latitude and longitude to the the dataframe:

In [27]:
# Installing geocoder
!pip install geocoder

     |████████████████████████████████| 98 kB 5.4 MB/s eta 0:00:011


In [28]:
# Import geocoder
import geocoder

Given link to a csv file that has the geographical coorinates of each postal code : http://cocl.us/Geospatial_data

So we read that csv using pandas and assign it to geo_data

In [29]:
geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Check the length of neighborhood data (**clean_df**) and geospatial data (**geo_data**) 

In [31]:
print(f'Length of Neighborhood data : {len(clean_df)}')
print(f'Length of Geospatial data : {len(geo_data)}')

Length of Neighborhood data : 103
Length of Geospatial data : 103


Check data types for each columns in neighborhood data and geospatial data

In [33]:
print(f'Data types of Neighborhood data : \n{clean_df.dtypes}\n')
print(f'Data types of Geospatial data : \n{geo_data.dtypes}')

Data types of Neighborhood data : 
Postal Code      object
Borough          object
Neighbourhood    object
dtype: object

Data types of Geospatial data : 
Postal Code     object
Latitude       float64
Longitude      float64
dtype: object


Join both dataframe on 'Postal Code' column

In [35]:
geo_data.set_index('Postal Code')

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


In [36]:
neighborhood_df = clean_df.join(geo_data.set_index('Postal Code'), on ='Postal Code', how='inner')
neighborhood_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Shape of neighborhood_df

In [37]:
neighborhood_df.shape

(103, 5)

In neighborhood_df we got 103 rows and now we have 5 columns adding latitude and longitude to the dataframe

## Problem 3

### Explore and cluster the neighborhoods in Toronto


- Get coordinate of Toronto using geopy.geocoders.Nominatim object
- Visualize map of Toronto with Folium library
- Get neighborhoods data using Foursquare API
- Find similarities of categories uising K-Means Clustering

In [38]:
# Find coordinates of Toronto
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


Visualizing map of Toronto

In [39]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for latitude, longitude, borough, neighbourhood in zip(neighborhood_df['Latitude'], neighborhood_df['Longitude'], neighborhood_df['Borough'], neighborhood_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(toronto_map)
    
toronto_map

Defining Foursquare API Credentials and Version

In [41]:
# @hidden cells
CLIENT_ID = 'L00XEGIHDO1OSWE2JBEV5WPRY4IILAHBLKKZ54WGRU51OTPU' # your Foursquare ID
CLIENT_SECRET = 'VZ2VJZJIZA1XAY4OXXOTMQHVMFUPVSPTBI21LUJSIDWE1K4U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L00XEGIHDO1OSWE2JBEV5WPRY4IILAHBLKKZ54WGRU51OTPU
CLIENT_SECRET:VZ2VJZJIZA1XAY4OXXOTMQHVMFUPVSPTBI21LUJSIDWE1K4U


Get a data about venues in Toronto using Foursquare API

In [49]:
# Function that return latitude, longitude, venues, and venue_categories in neighborhood_df
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    # create an empty list
    venues_list=[]
    
    # for loop that iterate through dataframe
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

        
    # Create pandas dataframe from venues_list
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

#### Get venues data for each neighborhood in Toronto

In [50]:
toronto_venues = getNearbyVenues(neighborhood_df['Neighbourhood'], neighborhood_df['Latitude'], neighborhood_df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

#### Check the size of the resulting dataframe

In [51]:
print(f'Shape of toronto_venues dataframe : {toronto_venues.shape}\n')
print('Head of toronto_venues dataframe : ')
toronto_venues.head()

Shape of toronto_venues dataframe : (1322, 5)

Head of toronto_venues dataframe : 


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,Portuguese Restaurant


#### Check how many venues were returned for each neighborhood

In [52]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
Neighborhood,,,,
Agincourt,5,5,5,5
"Alderwood, Long Branch",7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21
Bayview Village,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22
Berczy Park,30,30,30,30
"Birch Cliff, Cliffside West",4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16


#### How many unique categories can be curated from all the returned venues

In [53]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


### Using One Hot Encoding to create 237 categories as features/columns

In [54]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [55]:
# Shape of toronto_onehot
toronto_onehot.shape

(1322, 237)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [56]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [57]:
# Shape of toronto_grouped
toronto_grouped.shape

(96, 237)

#### Create a _pandas_ dataframe for each neighborhood with the top 10 most common venues

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Women's Store,Deli / Bodega,Drugstore,Donut Shop,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Coffee Shop,Sandwich Place,Pub,Dessert Shop,Dance Studio,Deli / Bodega,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Shopping Mall,Pizza Place,Sushi Restaurant,Ice Cream Shop,Restaurant,Deli / Bodega,Middle Eastern Restaurant,Chinese Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Greek Restaurant,Juice Bar,Café,Sushi Restaurant,Indian Restaurant,Pub,Restaurant


In [71]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Women's Store,Deli / Bodega,Drugstore,Donut Shop,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Coffee Shop,Sandwich Place,Pub,Dessert Shop,Dance Studio,Deli / Bodega,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Shopping Mall,Pizza Place,Sushi Restaurant,Ice Cream Shop,Restaurant,Deli / Bodega,Middle Eastern Restaurant,Chinese Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Greek Restaurant,Juice Bar,Café,Sushi Restaurant,Indian Restaurant,Pub,Restaurant
5,Berczy Park,Farmers Market,Beer Bar,Cocktail Bar,Seafood Restaurant,Coffee Shop,Liquor Store,Basketball Stadium,Jazz Club,Museum,Breakfast Spot
6,"Birch Cliff, Cliffside West",Café,College Stadium,General Entertainment,Skating Rink,Curling Ice,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
7,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Intersection,Bar,Italian Restaurant,Stadium,Restaurant,Climbing Gym
8,"Business reply mail Processing Centre, South C...",Auto Workshop,Comic Shop,Light Rail Station,Burrito Place,Butcher,Restaurant,Farmers Market,Fast Food Restaurant,Recording Studio,Skate Park
9,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Airport Terminal,Rental Car Location,Boutique


In [72]:
# Shape of neighborhoods_venues_sorted
neighborhoods_venues_sorted.shape

(96, 11)

# Clustering Neighborhood

Run _k_-means to cluster the neighborhood into 5 clusters.


In [73]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [74]:
# Renaming Neighborhoud columns from neighborhood_df to Neighborhood for joining dataframe
neighborhood_df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
neighborhood_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [75]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhood_df

# merge toronto_grouped with neighborhood_df to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Women's Store,Dance Studio,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,Pizza Place,Coffee Shop,French Restaurant,Hockey Arena,Portuguese Restaurant,Dim Sum Restaurant,Deli / Bodega,Department Store,Dessert Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Yoga Studio,French Restaurant,Dessert Shop,Spa,Event Space
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Gift Shop,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,Boutique,Accessories Store,Drugstore
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,College Auditorium,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Café,Portuguese Restaurant,Chinese Restaurant,Park


In [81]:
# Check na value for 'Cluster Label'
toronto_merged['Cluster Labels'].isna().sum()

3

In [82]:
# Drop NA
toronto_merged_clean = toronto_merged.dropna(subset=['Cluster Labels'])

In [83]:
# New shape of toronto_merged_clean
toronto_merged_clean.shape

(100, 16)

## Visualize the resulting clusters

In [86]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_clean['Latitude'], toronto_merged_clean['Longitude'], toronto_merged_clean['Neighborhood'], toronto_merged_clean['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine each Clusters

#### Cluster 1


In [87]:
toronto_merged_clean.loc[toronto_merged_clean['Cluster Labels'] == 0, toronto_merged_clean.columns[[1] + list(range(5, toronto_merged_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Park,Food & Drink Shop,Women's Store,Dance Studio,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
21,York,0.0,Park,Women's Store,Pool,Curling Ice,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
35,East York,0.0,Park,Convenience Store,Intersection,Women's Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
49,North York,0.0,Park,Construction & Landscaping,Bakery,Women's Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
61,Central Toronto,0.0,Park,Swim School,Bus Line,Women's Store,Deli / Bodega,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
64,York,0.0,Park,Women's Store,Curling Ice,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
66,North York,0.0,Park,Convenience Store,Women's Store,Dance Studio,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
85,Scarborough,0.0,Park,Playground,Bakery,Intersection,Women's Store,Deli / Bodega,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
91,Downtown Toronto,0.0,Park,Playground,Trail,Women's Store,Curling Ice,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


#### Cluster 2


In [88]:
toronto_merged_clean.loc[toronto_merged_clean['Cluster Labels'] == 1, toronto_merged_clean.columns[[1] + list(range(5, toronto_merged_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,Intersection,Pizza Place,Coffee Shop,French Restaurant,Hockey Arena,Portuguese Restaurant,Dim Sum Restaurant,Deli / Bodega,Department Store,Dessert Shop
2,Downtown Toronto,1.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Yoga Studio,French Restaurant,Dessert Shop,Spa,Event Space
3,North York,1.0,Clothing Store,Gift Shop,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,Boutique,Accessories Store,Drugstore
4,Downtown Toronto,1.0,Coffee Shop,College Auditorium,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Café,Portuguese Restaurant,Chinese Restaurant,Park
7,North York,1.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Art Gallery,Café,Restaurant,Italian Restaurant,Caribbean Restaurant,Asian Restaurant
8,East York,1.0,Pizza Place,Pharmacy,Bus Line,Bank,Intersection,Athletics & Sports,Breakfast Spot,Gastropub,Pet Store,Gym / Fitness Center
9,Downtown Toronto,1.0,Café,Clothing Store,Theater,Coffee Shop,Ramen Restaurant,Sandwich Place,Fast Food Restaurant,Steakhouse,Bakery,College Rec Center
10,North York,1.0,Park,Pizza Place,Japanese Restaurant,Pub,Bakery,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center
12,Scarborough,1.0,Bar,Construction & Landscaping,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
13,North York,1.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Art Gallery,Café,Restaurant,Italian Restaurant,Caribbean Restaurant,Asian Restaurant


#### Cluster 3

In [89]:
toronto_merged_clean.loc[toronto_merged_clean['Cluster Labels'] == 2, toronto_merged_clean.columns[[1] + list(range(5, toronto_merged_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,2.0,Martial Arts School,Women's Store,Dance Studio,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


#### Cluster 4

In [90]:
toronto_merged_clean.loc[toronto_merged_clean['Cluster Labels'] == 3, toronto_merged_clean.columns[[1] + list(range(5, toronto_merged_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,3.0,Print Shop,Women's Store,Curling Ice,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


#### Cluster 5

In [91]:
toronto_merged_clean.loc[toronto_merged_clean['Cluster Labels'] == 4, toronto_merged_clean.columns[[1] + list(range(5, toronto_merged_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4.0,Fast Food Restaurant,Women's Store,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


## Finally finished Clustering Toronto with 5 Clusters using K-Means Clustering

## Author : Muhammad Adisatriyo Pratama
## References : [Coursera Applied Data Science Capstone](https://www.coursera.org/learn/applied-data-science-capstone)

### December 2020